In [1]:
import pandas as pd

inter_df = pd.read_csv('ratings.csv', header=0, sep=',')

In [3]:
inter_df['movieId'].groupby(inter_df['userId']).count().mean()

102.21969030893572

In [4]:
inter_df['userId'].groupby(inter_df['movieId']).count().mean()

406.4460409183195

In [6]:
len(inter_df['userId'].unique())

330975

In [7]:
len(inter_df['movieId'].unique())

83239

In [10]:
import random
import os
import re
import sys
work_dir = re.search('(.*LM4REC).*', os.getcwd(), re.IGNORECASE).group(1)
sys.path.append(work_dir)
sys.path.append(os.path.join(work_dir, 'RecStudio'))
from RecStudio.recstudio.utils import seed_everything

seed_everything(42)

num_users = len(inter_df['userId'].unique())
rand_users = []
for _ in range(100000):
    while True:
        rand_user = random.randint(0, num_users - 1)
        if rand_user not in rand_users:
            rand_users.append(rand_user)
            break

In [12]:
inter_df['userId'].unique()

array([     1,      2,      3, ..., 330973, 330974, 330975])

In [13]:
inter_df['userId'].unique().max()

330975

In [14]:
inter_df['userId'].unique().min()

1

In [15]:
rand_users = [_+ 1 for _ in rand_users]

In [17]:
rand_users = set(rand_users)

In [18]:
keep = inter_df['userId'].map(lambda x : True if x in rand_users else False)

In [20]:
inter_df[keep]

,userId,movieId,rating,timestamp
183,4,260,5.0,1442455519
184,4,318,5.0,1442455559
185,4,356,5.0,1442455449
186,4,595,4.5,1442456131
187,4,858,3.0,1442455657
...,...,...,...,...
33831682,330973,7149,4.0,1383438153
33831683,330973,7153,3.0,1383438424
33831684,330973,31685,4.0,1383438076
33831685,330973,58559,4.5,1383438328


In [22]:
sum(inter_df[keep]['movieId'].groupby(inter_df[keep]['userId']).count() > 5)

90316

In [23]:
sum(inter_df[keep]['userId'].groupby(inter_df[keep]['movieId']).count() > 5)

23948

In [24]:
sampled_inter_df = inter_df[keep]

In [26]:
sampled_inter_df.reset_index(drop=True, inplace=True)

In [27]:
sampled_inter_df

,userId,movieId,rating,timestamp
0,4,260,5.0,1442455519
1,4,318,5.0,1442455559
2,4,356,5.0,1442455449
3,4,595,4.5,1442456131
4,4,858,3.0,1442455657
...,...,...,...,...
10223175,330973,7149,4.0,1383438153
10223176,330973,7153,3.0,1383438424
10223177,330973,31685,4.0,1383438076
10223178,330973,58559,4.5,1383438328


In [28]:
import scipy.sparse as ssp
import warnings
import numpy as np

item_list = sampled_inter_df['movieId']
item_idx_list, items = pd.factorize(item_list)
user_list = sampled_inter_df['userId']
user_idx_list, users = pd.factorize(user_list)
warnings.simplefilter('ignore', ssp.SparseEfficiencyWarning)
user_item_mat = ssp.csc_matrix(
    (np.ones_like(user_idx_list), (user_idx_list, item_idx_list)))
cols = np.arange(items.size)
rows = np.arange(users.size)
while(True): # TODO: only delete users/items in inter_feat, users/items in user/item_feat should also be deleted.
    m, n = user_item_mat.shape
    col_sum = np.squeeze(user_item_mat.sum(axis=0).A)
    col_ind = col_sum >= 5
    col_count = np.count_nonzero(col_ind)
    if col_count > 0:
        cols = cols[col_ind]
        user_item_mat = user_item_mat[:, col_ind]
    row_sum = np.squeeze(user_item_mat.sum(axis=1).A)
    row_ind = row_sum >= 5
    row_count = np.count_nonzero(row_ind)
    if row_count > 0:
        rows = rows[row_ind]
        user_item_mat = user_item_mat[row_ind, :]
    if col_count == n and row_count == m:
        break
    else:
        pass
#
keep_users = set(users[rows])
keep_items = set(items[cols])
keep = user_list.isin(keep_users)
keep &= item_list.isin(keep_items)
re_sampled_inter_df = sampled_inter_df[keep]
re_sampled_inter_df.reset_index(drop=True, inplace=True)

In [29]:
re_sampled_inter_df

,userId,movieId,rating,timestamp
0,4,260,5.0,1442455519
1,4,318,5.0,1442455559
2,4,356,5.0,1442455449
3,4,595,4.5,1442456131
4,4,858,3.0,1442455657
...,...,...,...,...
10146325,330973,7149,4.0,1383438153
10146326,330973,7153,3.0,1383438424
10146327,330973,31685,4.0,1383438076
10146328,330973,58559,4.5,1383438328


In [45]:
len(re_sampled_inter_df.userId.unique())

92910

In [47]:
re_sampled_inter_df['movieId'].groupby(re_sampled_inter_df['userId']).count().mean()

109.2060058120762

In [44]:
re_sampled_inter_df.to_csv('sampled_ratings.csv', sep=',', index=False)

In [32]:
keep_movie_id = set(re_sampled_inter_df['movieId'].to_list())

In [30]:
item_link_df = pd.read_csv('links.csv', header=0, sep=',')

In [31]:
item_link_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
86532,288967,14418234,845861.0
86533,288971,11162178,878958.0
86534,288975,70199,150392.0
86535,288977,23050520,1102551.0


In [33]:
keep_movie = item_link_df['movieId'].map(lambda x: True if x in keep_movie_id else False)

In [34]:
sampled_item_link_df = item_link_df[keep_movie]

In [35]:
sampled_item_link_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
86303,288167,12263384,697843.0
86320,288265,9603212,575264.0
86321,288269,15671028,884605.0
86338,288311,19500164,961323.0


In [42]:
sampled_item_link_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26083 entries, 0 to 86439
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  26083 non-null  int64  
 1   imdbId   26083 non-null  int64  
 2   tmdbId   26056 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 815.1 KB


In [ ]:
sampled_item_link_df.to_csv('sampled_links.csv', sep=',', index=False)